In [1]:
import os
import cv2
import time
import sklearn
from sklearn.cluster import KMeans
from scipy.spatial import distance
import numpy as np
import pickle
from PIL import Image
## extract the descriptors from all images. ##
sift = cv2.xfeatures2d.SIFT_create()

In [2]:
def bag_of_words(centroids, img_descriptors):
    n_centroids = centroids.shape[0]  # number of centroids found with the KMeans clustering #100
    n_descriptors = img_descriptors.shape[0]  # number of descriptors extracted from the image #200
    
    # initialization of the bag of words (BoW) vector
    # Note that the BoW vector has length equal to the number of cluster centroids
    # The cluster centroids are indeed our visual words, and the BoW will be the histogram of these words found in the given image
    bow_vector = np.zeros(n_centroids)  
    
    for i in range(n_descriptors):
        for j in range(n_centroids):
            if img_descriptors[i][j]==True: #if the feature is in the image (true in img_descriptor)
                bow_vector[j]+=1            #bow_vector.shape => 100 (for each image)
    return bow_vector

In [3]:
import json
import numpy as np
import matplotlib.pyplot as plt

## RETRIEVE IMAGES ##

def retrieve_images(map_bow_vectors, query_bow):
    n_map_bow_vectors = map_bow_vectors.shape[0]
    bow_distances = np.zeros(n_map_bow_vectors)
    most_similar = None  
    
    for i in range(n_map_bow_vectors):
        for j in range(len(query_bow)):
            
            bow_distances[i] += distance.euclidean(map_bow_vectors[i][j], query_bow[j])        
            #bow_distances[i] +=distance.minkowski(map_bow_vectors[i][j], query_bow[j])
            #bow_distances[i] += distance.cosine(map_bow_vectors[i][j], query_bow[j])
            #bow_distances[i] += distance.cityblock(map_bow_vectors[i][j], query_bow[j]) #manhattan
              
    most_similar=np.argsort(bow_distances)
    most_similar=most_similar[:10]
        
    return most_similar


# Retrieve the most similar images to query image 87 (index 87-1=86)
def search_bagofword(image_path):
    start = time.time()
    bow_map_images = np.load("./bag_of_words/bow_map_images.npy")
    centroids = np.load("./bag_of_words/centroids.npy")
    f = open('./bag_of_words/mylist1.txt','r')
    names = json.loads(f.read())
    f.close()
    f = open('./bag_of_words/mylist2.txt','r')
    labels = json.loads(f.read())
    f.close()
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    (kps, query_img_descriptors) = sift.detectAndCompute(gray, None)
    
    with open("./bag_of_words/kmeanmodel.pkl","rb") as f:
        kmeans2=pickle.load(f)
    
    bow = bag_of_words(centroids, query_img_descriptors)

    # Retrieve the indices of the top-10 similar images from the map
    retrieved_images = retrieve_images(bow_map_images, bow)

    #print('Indices of similar images retrieved: ', retrieved_images)
    # Indices of the relevant map images for the query: we have the relevance judgements (Ground truth)
    #relevant_images = np.where(sim[86, :] == 1)[0]
    #print('Indices of relevant images (given in the GT relevance judgements): ', relevant_images)
    
    s_img = image_path
    s_image = Image.open(s_img)
    
    with open('./search_results/result1.jpg', 'wb') as f:
        s_image.save(f, "JPEG", quality=85)
    
    s1 = "images/" + names[retrieved_images[0]]
    
    s_img = s1
    s_image = Image.open(s_img)
    
    with open('./search_results/result2.jpg', 'wb') as f:
        s_image.save(f, "JPEG", quality=85)
    
    s2 = "images/" + names[retrieved_images[1]]
    
    s_img = s2
    s_image = Image.open(s_img)
    
    with open('./search_results/result3.jpg', 'wb') as f:
        s_image.save(f, "JPEG", quality=85)
    
    end = time.time()
        
    plt.figure(figsize=(17,10))
    plt.subplot(1,3,1)
    plt.title('queued')
    plt.imshow(plt.imread(image_path))
    plt.axis("off")
    
    
        
    plt.subplot(1,3,2)
    plt.title('first retieved-'+labels[names[retrieved_images[0]]])
    plt.imshow(plt.imread(s1))
    plt.axis("off")
        
    plt.subplot(1,3,3)
    plt.title('second retieved-'+labels[names[retrieved_images[1]]])
    plt.imshow(plt.imread(s2))
    plt.axis("off")
        
    print()
    print("time taken to search:",round(end-start,1),"sec")
    list2 = []
    list2.append(labels[names[retrieved_images[0]]])
    list2.append(labels[names[retrieved_images[1]]])
    list2.append("Time taken to search : "+ str(round(end-start,1)) + " sec")
    return list2

## search function

In [4]:
def search_image(image_path):
    img = image_path
    image = Image.open(img)
    width = image.width
    height = image.height

    if width>300:
        basewidth = 300
        wpercent = (basewidth / float(image.size[0]))
        hsize = int((float(image.size[1]) * float(wpercent)))
        image = image.resize((basewidth, hsize), Image.ANTIALIAS)
            
    if height>300:
        baseheight = 300
        hpercent = (baseheight / float(image.size[1]))
        wsize = int((float(image.size[0]) * float(hpercent)))
        image = image.resize((wsize, baseheight), Image.ANTIALIAS)

    with open('./search_images/temp.jpg', 'wb') as f:
        image.save(f, "JPEG", quality=85)
    
    dmlist = search_bagofword('./search_images/temp.jpg')
    return dmlist

In [6]:
#dm = search_image('./search_images/images8.jpg')